# TFX Components Walk-through

## Learning objectives

1.  Develop a high level understanding of TFX pipeline components.
2.  Learn how to use a TFX Interactive Context for prototype development of TFX pipelines.
3.  Work with the Tensorflow Data Validation (TFDV) library to check and analyze input data.
4.  Utilize the Tensorflow Transform (TFT) library for scalable data preprocessing and feature transformations.
5.  Employ the Tensorflow Model Analysis (TFMA) library for model evaluation.

## Introduction

In this notebook, you will work with the [Covertype Data Set](https://github.com/jarokaz/mlops-labs/blob/master/datasets/covertype/README.md) and use TFX to analyze, understand, and pre-process the dataset and train, analyze, validate, and deploy a multi-class classification model to predict the type of forest cover from cartographic features.

You will utilize  **TFX Interactive Context** to work with the TFX components interactivelly in a Jupyter notebook environment. Working in an interactive notebook is useful when doing initial data exploration, experimenting with models, and designing ML pipelines. You should be aware that there are differences in the way interactive notebooks are orchestrated, and how they access metadata artifacts. In a production deployment of TFX on GCP, you will use an orchestrator such as Kubeflow Pipelines, or Cloud Composer. In an interactive mode, the notebook itself is the orchestrator, running each TFX component as you execute the notebook cells. In a production deployment, ML Metadata will be managed in a scalabe database like MySQL, and artifacts in apersistent store such as Google Cloud Storage. In an interactive mode, both properties and payloads are stored in a local file system of the Jupyter host.

**Setup Note**:

Currently, TFMA visualizations do not render properly in JupyterLab. It is recommended to run this notebook in Jupyter Classic Notebook. To switch to Classic Notebook select **Launch Classic Notebook** from the **Help** menu.

In [1]:
import absl
import os
import tempfile
import time

import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tfx

from pprint import pprint
from tensorflow_metadata.proto.v0 import schema_pb2, statistics_pb2, anomalies_pb2
from tensorflow_transform.tf_metadata import schema_utils
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import InfraValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components import Tuner
from tfx.dsl.components.base import executor_spec
from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.components.trainer import executor as trainer_executor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import infra_validator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import HyperParameters
from tfx.types.standard_artifacts import ModelBlessing
from tfx.types.standard_artifacts import InfraBlessing

**Note**: this lab was developed and tested with the following TF ecosystem package versions:

`Tensorflow Version: 2.3.1`  
`TFX Version: 0.25.0`  
`TFDV Version: 0.25.0`  
`TFMA Version: 0.25.0`

If you encounter errors with the above imports (e.g. TFX component not found), check your package versions in the cell below.

In [2]:
print("Tensorflow Version:", tf.__version__)
print("TFX Version:", tfx.__version__)
print("TFDV Version:", tfdv.__version__)
print("TFMA Version:", tfma.VERSION_STRING)

absl.logging.set_verbosity(absl.logging.INFO)

Tensorflow Version: 2.3.4
TFX Version: 0.25.0
TFDV Version: 0.25.0
TFMA Version: 0.25.0


If the versions above do not match, update your packages in the current Jupyter kernel below. The default `%pip` package installation location is not on your system installation PATH; use the command below to append the local installation path to pick up the latest package versions. Note that you may also need to restart your notebook kernel to pick up the specified package versions and re-run the imports cell above before proceeding with the lab.

In [3]:
os.environ['PATH'] += os.pathsep + '/home/jupyter/.local/bin'

(Optional) Run the cell below only if your package versions do not match the lab defaults.

In [4]:
%pip install --upgrade --user tensorflow==2.3.1
%pip install --upgrade --user tfx==0.25.0
%pip install --upgrade --user tensorflow_data_validation==0.25.0
%pip install --upgrade --user tensorflow_model_analysis==0.25.0

     |████████████████████████████████| 320.4 MB 10 kB/s              .4 MB 18.8 MB/s eta 0:00:16██▌                        | 75.0 MB 9.6 MB/s eta 0:00:26 ��████▍                    | 113.8 MB 9.8 MB/s eta 0:00:22     | 129.5 MB 33.9 MB/s eta 0:00:06��████████▏                  | 131.9 MB 33.9 MB/s eta 0:00:06█████████                  | 141.1 MB 270 kB/s eta 0:11:03 98 kB/s eta 0:03:37 ███▉                | 158.1 MB 269 kB/s eta 0:10:02  |████████████████▍               | 164.1 MB 785 kB/s eta 0:03:20  | 174.1 MB 750 kB/s eta 0:03:15 ��███████▎             | 183.5 MB 749 kB/s eta 0:03:03   | 205.2 MB 665 kB/s eta 0:02:53 ��█████████████████████         | 231.1 MB 792 kB/s eta 0:01:53 B 792 kB/s eta 0:01:48 ��████████▋   | 286.6 MB 46.4 MB/s eta 0:00:01��███████████████████▎  | 293.3 MB 46.4 MB/s eta 0:00:01 |██████████████████████████████▍ | 303.9 MB 8.4 MB/s eta 0:00:02  MB 62.3 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages th

**Restart the kernel by using Kernel > Restart kernel > Restart.**

## Configure lab settings

Set constants, location paths and other environment settings. 

In [1]:
import absl
import os
import tempfile
import time

import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tfx

from pprint import pprint
from tensorflow_metadata.proto.v0 import schema_pb2, statistics_pb2, anomalies_pb2
from tensorflow_transform.tf_metadata import schema_utils
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import InfraValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components import Tuner
from tfx.dsl.components.base import executor_spec
from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.components.trainer import executor as trainer_executor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import infra_validator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import HyperParameters
from tfx.types.standard_artifacts import ModelBlessing
from tfx.types.standard_artifacts import InfraBlessing

2023-10-20 14:55:48.640160: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2023-10-20 14:55:48.640222: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
ARTIFACT_STORE = os.path.join(os.sep, 'home', 'jupyter', 'artifact-store')
SERVING_MODEL_DIR=os.path.join(os.sep, 'home', 'jupyter', 'serving_model')
DATA_ROOT = 'gs://cloud-training/OCBL203/workshop-datasets'

## Creating Interactive Context

TFX Interactive Context allows you to create and run TFX Components in an interactive mode. It is designed to support experimentation and development in a Jupyter Notebook environment. It is an experimental feature and major changes to interface and functionality are expected. When creating the interactive context you can specifiy the following parameters:
- `pipeline_name` - Optional name of the pipeline for ML Metadata tracking purposes. If not specified, a name will be generated for you.
- `pipeline_root` - Optional path to the root of the pipeline's outputs. If not specified, an ephemeral temporary directory will be created and used.
- `metadata_connection_config` - Optional `metadata_store_pb2.ConnectionConfig` instance used to configure connection to a ML Metadata connection. If not specified, an ephemeral SQLite MLMD connection contained in the pipeline_root directory with file name "metadata.sqlite" will be used.


In [3]:
PIPELINE_NAME = 'tfx-covertype-classifier'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME, time.strftime("%Y%m%d_%H%M%S"))
os.makedirs(PIPELINE_ROOT, exist_ok=True)

context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=None)

## Ingesting data using ExampleGen

In any ML development process the first step  is to ingest the training and test datasets. The `ExampleGen` component ingests data into a TFX pipeline. It consumes external files/services to generate a set file files in the `TFRecord` format,  which will be used by other TFX components. It can also shuffle the data and split into an arbitrary number of partitions.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/ExampleGen.png width="300">

### Configure and run CsvExampleGen

In this exercise, you use the `CsvExampleGen` specialization of `ExampleGen` to ingest CSV files from a GCS location and emit them as `tf.Example` records for consumption by downstream TFX pipeline components. Your task is to configure the component to create 80-20 `train` and `eval` splits. *Hint: review the [ExampleGen proto](https://github.com/tensorflow/tfx/blob/master/tfx/proto/example_gen.proto#L78) definition to split your data with hash buckets.*

In [4]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        # TODO: Your code to configure train data split
        # TODO: Your code to configure eval data split
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)        
    ]))

example_gen = tfx.components.CsvExampleGen(
    input_base=DATA_ROOT,
    output_config=output_config)

In [5]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:5611885,xor_checksum:1657792105,sum_checksum:1657792105"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

### Examine the ingested data

In [6]:
examples_uri = example_gen.outputs['examples'].get()[0].uri
tfrecord_filenames = [os.path.join(examples_uri, 'train', name)
                      for name in os.listdir(os.path.join(examples_uri, 'train'))]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
  example = tf.train.Example()
  example.ParseFromString(tfrecord.numpy())
  for name, feature in example.features.feature.items():
    if feature.HasField('bytes_list'):
        value = feature.bytes_list.value
    if feature.HasField('float_list'):
        value = feature.float_list.value
    if feature.HasField('int64_list'):
        value = feature.int64_list.value
    print('{}: {}'.format(name, value))
  print('******')

2023-10-20 15:01:15.903786: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-10-20 15:01:15.903861: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-20 15:01:15.903899: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tfx-on-googlecloud): /proc/driver/nvidia/version does not exist
2023-10-20 15:01:15.951625: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 15:01:16.334303: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 220

Aspect: [183]
Slope: [9]
Horizontal_Distance_To_Hydrology: [648]
Vertical_Distance_To_Hydrology: [101]
Hillshade_9am: [223]
Cover_Type: [1]
Wilderness_Area: [b'Commanche']
Soil_Type: [b'C7757']
Hillshade_3pm: [157]
Horizontal_Distance_To_Roadways: [757]
Elevation: [3142]
Horizontal_Distance_To_Fire_Points: [1871]
Hillshade_Noon: [247]
******
Hillshade_3pm: [105]
Soil_Type: [b'C2704']
Slope: [16]
Elevation: [1967]
Hillshade_9am: [245]
Vertical_Distance_To_Hydrology: [9]
Horizontal_Distance_To_Fire_Points: [451]
Cover_Type: [2]
Wilderness_Area: [b'Cache']
Aspect: [124]
Hillshade_Noon: [227]
Horizontal_Distance_To_Roadways: [124]
Horizontal_Distance_To_Hydrology: [60]
******


## Generating statistics using StatisticsGen

The `StatisticsGen`  component generates data statistics that can be used by other TFX components. StatisticsGen uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started). `StatisticsGen` generate statistics for each split in the `ExampleGen` component's output. In our case there two splits: `train` and `eval`.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/StatisticsGen.png width="200">

### Configure and  run the `StatisticsGen` component

In [7]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

In [8]:
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

### Visualize statistics

The generated statistics can be visualized using the `tfdv.visualize_statistics()` function from the [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library or using a utility method of the `InteractiveContext` object. In fact, most of the artifacts generated by the TFX components can be visualized using `InteractiveContext`.

In [9]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


## Infering data schema using SchemaGen

Some TFX components use a description input data called a schema. The schema is an instance of `schema.proto`. It can specify data types for feature values, whether a feature has to be present in all examples, allowed value ranges, and other properties. `SchemaGen` automatically generates the schema by inferring types, categories, and ranges from data statistics. The auto-generated schema is best-effort and only tries to infer basic properties of the data. It is expected that developers review and modify it as needed. `SchemaGen` uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

The `SchemaGen` component generates the schema using the statistics for the `train` split. The statistics for other splits are ignored.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/SchemaGen.png width="200">

### Configure and run the `SchemaGen` components

In [10]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)

In [11]:
context.run(schema_gen)

2023-10-20 15:02:28.298333: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

### Visualize the inferred schema

In [12]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,-
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-


/home/jupyter/.local/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## Updating the auto-generated schema

In most cases the auto-generated schemas must be fine-tuned manually using insights from data exploration and/or domain knowledge about the data. For example, you know that in the `covertype` dataset there are seven types of forest cover (coded using 1-7 range) and that the value of the `Slope` feature should be in the 0-90 range. You can manually add these constraints to the auto-generated schema by setting the feature domain.

### Load the auto-generated schema proto file

In [13]:
schema_proto_path = '{}/{}'.format(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_proto_path)

### Modify the schema

You can use the protocol buffer APIs to modify the schema. 

**Hint**: Review the [TFDV library API documentation](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/set_domain) on setting a feature's domain. You can use the protocol buffer APIs to modify the schema. Review the [Tensorflow Metadata proto definition](https://github.com/tensorflow/metadata/blob/master/tensorflow_metadata/proto/v0/schema.proto#L405) for configuration options.

In [14]:
# TODO: Your code to restrict the categorical feature Cover_Type between the values of 0 and 6.
# TODO: Your code to restrict the numeric feature Slope between 0 and 90.
tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=0, max=6, is_categorical=True))
tfdv.set_domain(schema, 'Slope',  schema_pb2.IntDomain(name='Slope', min=0, max=90))

tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,"[0,6]"
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-


/home/jupyter/.local/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


#### Save the updated schema 

In [15]:
schema_dir = os.path.join(ARTIFACT_STORE, 'schema')
tf.io.gfile.makedirs(schema_dir)
schema_file = os.path.join(schema_dir, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

feature {
  name: "Aspect"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Cover_Type"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  int_domain {
    name: "Cover_Type"
    min: 0
    max: 6
    is_categorical: true
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Elevation"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Hillshade_3pm"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Hillshade_9am"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Hillshade_Noon"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Horizontal_Distance_To_Fire_P

## Importing the updated schema using ImporterNode

The `ImporterNode` component allows you to import an external artifact, including the schema file, so it can be used by other TFX components in your workflow. 


### Configure and run the `ImporterNode` component

In [16]:
schema_importer = ImporterNode(
    instance_name='Schema_Importer',
    source_uri=schema_dir,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=False)

In [17]:
context.run(schema_importer)

ExecutionResult(
    component_id: ImporterNode.Schema_Importer
    execution_id: 4
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 4
        type_id: 9
        uri: "/home/jupyter/artifact-store/schema"
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

### Visualize the imported schema

In [18]:
context.show(schema_importer.outputs['result'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,"[0,6]"
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-


/home/jupyter/.local/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## Validating data with ExampleValidator

The `ExampleValidator` component identifies anomalies in data.  It identifies anomalies by comparing data statistics computed by the `StatisticsGen` component against a schema generated by `SchemaGen` or imported by `ImporterNode`.

`ExampleValidator` can detect different classes of anomalies. For example it can:

- perform validity checks by comparing data statistics against a schema 
- detect training-serving skew by comparing training and serving data.
- detect data drift by looking at a series of data.


The `ExampleValidator` component validates the data in the `eval` split only. Other splits are ignored. 

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/ExampleValidator.png width="350">

### Configure and run the `ExampleValidator` component


In [19]:
# TODO: Complete ExampleValidator
# Hint: review the visual above and review the documentation on ExampleValidator's inputs and outputs: 
# https://www.tensorflow.org/tfx/guide/exampleval
# Make sure you use the output of the schema_importer component created above.

example_validator = ExampleValidator(
    instance_name="Data_Validation",    
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result'])

In [20]:
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator.Data_Validation
    execution_id: 5
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 5
        type_id: 12
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/ExampleValidator.Data_Validation/anomalies/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator.Data_Validation"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 12
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

### Examine the output of `ExampleValidator`

The output artifact of the ExampleValidator is the `anomalies.pbtxt` file describing an anomalies_pb2.Anomalies protobuf.

In [21]:
train_uri = example_validator.outputs['anomalies'].get()[0].uri
train_anomalies_filename = os.path.join(train_uri, "train/anomalies.pbtxt")
!cat $train_anomalies_filename

baseline {
  feature {
    name: "Aspect"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "Cover_Type"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    int_domain {
      name: "Cover_Type"
      min: 0
      max: 6
      is_categorical: true
    }
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "Elevation"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "Hillshade_3pm"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "Hillshade_9am"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "Hillshade_Noon"
    value_cou

### Visualize validation results

The file `anomalies.pbtxt` can be visualized using `context.show`.

In [22]:
context.show(example_validator.outputs['output'])

In our case no anomalies were detected in the `eval` split.

For a detailed deep dive into data validation and schema generation refer to the `lab-31-tfdv-structured-data` lab.

## Preprocessing data with Transform

The `Transform` component performs data transformation and feature engineering. The `Transform` component consumes `tf.Examples` emitted from the `ExampleGen` component and emits the transformed feature data and the `SavedModel` graph that was used to process the data. The emitted `SavedModel`  can then be used by serving components to make sure that the same data pre-processing logic is applied at training and serving.

The `Transform` component requires more code than many other components because of the arbitrary complexity of the feature engineering that you may need for the data and/or model that you're working with. It requires code files to be available which define the processing needed.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Transform.png width="400">

### Define the pre-processing module

To configure `Trainsform`, you need to encapsulate your pre-processing code in the Python `preprocessing_fn` function and save it to a  python module that is then provided to the Transform component as an input. This module will be loaded by transform and the `preprocessing_fn` function will be called when the `Transform` component runs.

In most cases, your implementation of the `preprocessing_fn` makes extensive use of [TensorFlow Transform](https://www.tensorflow.org/tfx/guide/tft) for performing feature engineering on your dataset.

In [23]:
TRANSFORM_MODULE = 'preprocessing.py'
!cat {TRANSFORM_MODULE}

# Copyright 2021 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Covertype preprocessing.
This file defines a template for TFX Transform component.
"""

import tensorflow as tf
import tensorflow_transform as tft

import features

def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 

### Configure and run the `Transform` component.

In [24]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=TRANSFORM_MODULE)

In [25]:
context.run(transform)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


2023-10-20 15:06:48.958459: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
2023-10-20 15:06:48.963234: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


...

value: "\n\013\n\tConst_5:0\022\tSoil_Type"



value: "\n\013\n\tConst_5:0\022\tSoil_Type"



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Transform/transform_graph/6"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "TransformGraph"
        )]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 7
        type_id: 5
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Transform/transformed_examples/6"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 8
        type_id: 15
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Transform/updated_analyzer_cache/6"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 15
        name: "TransformCache"
        )]
        ))

### Examine the `Transform` component's outputs

The Transform component has 2 outputs:

- `transform_graph` - contains the graph that can perform the preprocessing operations (this graph will be included in the serving and evaluation models).
- `transformed_examples` - contains the preprocessed training and evaluation data.

Take a peek at the `transform_graph` artifact: it points to a directory containing 3 subdirectories:

In [26]:
os.listdir(transform.outputs['transform_graph'].get()[0].uri)

['transform_fn', 'metadata', 'transformed_metadata']

And the `transform.examples` artifact

In [27]:
os.listdir(transform.outputs['transformed_examples'].get()[0].uri)

['train', 'eval']

In [28]:
transform_uri = transform.outputs['transformed_examples'].get()[0].uri
tfrecord_filenames = [os.path.join(transform_uri,  'train', name)
                      for name in os.listdir(os.path.join(transform_uri, 'train'))]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
  example = tf.train.Example()
  example.ParseFromString(tfrecord.numpy())
  for name, feature in example.features.feature.items():
    if feature.HasField('bytes_list'):
        value = feature.bytes_list.value
    if feature.HasField('float_list'):
        value = feature.float_list.value
    if feature.HasField('int64_list'):
        value = feature.int64_list.value
    print('{}: {}'.format(name, value))
  print('******')

Cover_Type_xf: [1]
Hillshade_9am_xf: [0.4025527238845825]
Soil_Type_xf: [3]
Hillshade_Noon_xf: [1.1993097066879272]
Aspect_xf: [0.24982304871082306]
Horizontal_Distance_To_Hydrology_xf: [1.7856396436691284]
Vertical_Distance_To_Hydrology_xf: [0.950199544429779]
Elevation_xf: [0.651531994342804]
Wilderness_Area_xf: [1]
Slope_xf: [-0.6797884106636047]
Horizontal_Distance_To_Fire_Points_xf: [-0.08278580754995346]
Hillshade_3pm_xf: [0.38384687900543213]
Horizontal_Distance_To_Roadways_xf: [-1.0252416133880615]
******
Horizontal_Distance_To_Roadways_xf: [-1.4311398267745972]
Hillshade_3pm_xf: [-0.9800227880477905]
Soil_Type_xf: [19]
Horizontal_Distance_To_Fire_Points_xf: [-1.1525152921676636]
Horizontal_Distance_To_Hydrology_xf: [-0.9832831621170044]
Wilderness_Area_xf: [2]
Slope_xf: [0.25757989287376404]
Hillshade_Noon_xf: [0.1884450614452362]
Cover_Type_xf: [2]
Elevation_xf: [-3.5473971366882324]
Hillshade_9am_xf: [1.2273873090744019]
Aspect_xf: [-0.2777789831161499]
Vertical_Distance_To_

## Train your TensorFlow model with the `Trainer` component

The `Trainer` component trains a model using TensorFlow.

`Trainer` takes:

- tf.Examples used for training and eval.
- A user provided module file that defines the trainer logic.
- A data schema created by `SchemaGen` or imported by `ImporterNode`.
- A proto definition of train args and eval args.
- An optional transform graph produced by upstream Transform component.
- An optional base models used for scenarios such as warmstarting training.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Trainer.png width="400">


### Define the trainer module

To configure `Trainer`, you need to encapsulate your training code in a Python module that is then provided to the `Trainer` as an input. 


In [29]:
TRAINER_MODULE_FILE = 'model.py'
!cat {TRAINER_MODULE_FILE}

# Copyright 2021 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Covertype Keras WideDeep Classifier."""

import functools
import absl
import os
from typing import List, Text

import kerastuner
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import DataAc

### Create and run the Trainer component

As of the 0.25.0 release of TFX, the `Trainer` component only supports passing a single field - `num_steps` - through the `train_args` and `eval_args` arguments. 

In [30]:
trainer = Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=TRAINER_MODULE_FILE,
    transformed_examples=transform.outputs.transformed_examples,
    schema=schema_importer.outputs.result,
    transform_graph=transform.outputs.transform_graph,
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1000))

In [31]:
context.run(trainer)

2023-10-20 15:08:49.758024: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
2023-10-20 15:08:49.765353: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
2023-10-20 15:08:50.627365: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.


   1/5000 [..............................] - ETA: 0s - loss: 1.9306 - sparse_categorical_accuracy: 0.1719WARNING:tensorflow:From /home/jupyter/.local/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


2023-10-20 15:08:54.761987: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


...

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


2023-10-20 15:09:20.707157: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Trainer/model/7/serving_model_dir/assets


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Trainer/model/7/serving_model_dir/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 9
        type_id: 17
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Trainer/model/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "Model"
        )]
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 10
        type_id: 18
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Trainer/model_run/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "ModelRun"
        )]
        ))

### Analyzing training runs with TensorBoard

In this step you will analyze the training run with [TensorBoard.dev](https://blog.tensorflow.org/2019/12/introducing-tensorboarddev-new-way-to.html). `TensorBoard.dev` is a managed service that enables you to easily host, track and share your ML experiments.

#### Retrieve the location of TensorBoard logs

Each model run's train and eval metric logs are written to the `model_run` directory by the Tensorboard callback defined in `model.py`.

In [32]:
logs_path = trainer.outputs['model_run'].get()[0].uri
print(logs_path)

/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Trainer/model_run/7


#### Upload the logs and start TensorBoard.dev

1. Open a new JupyterLab terminal window

2. From the terminal window, execute the following command
```
tensorboard dev upload --logdir [YOUR_LOGDIR]
```

Where [YOUR_LOGDIR] is an URI retrieved by the previous cell.

You will be asked to authorize `TensorBoard.dev` using your Google account. If you don't have a Google account or you don't want to authorize `TensorBoard.dev` you can skip this exercise.

After the authorization process completes, follow the link provided to view your experiment.

## Tune your model's hyperparameters with the `Tuner` component

The `Tuner` component makes use of the Python [KerasTuner](https://keras-team.github.io/keras-tuner/) API to tune your model's hyperparameters. It tighty integrates with the `Transform` and `Trainer` components for model hyperparameter tuning in continuous training pipelines as well as advanced use cases such as feature selection, feature engineering, and model architecture search.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Tuner_Overview.png width="400">

`Tuner` takes:

*  A user provided module file (or module fn) that defines the tuning logic, including model definition, hyperparameter search space, objective etc.
*  tf.Examples used for training and eval.
*  Protobuf definition of train args and eval args.
  * (Optional) Protobuf definition of tuning args.
  * (Optional) transform graph produced by an upstream Transform component.
  * (Optional) A data schema created by a SchemaGen pipeline component and optionally altered by the developer.
  
<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Tuner.png width="400">

With the given data, model, and objective, `Tuner` tunes the hyperparameters and emits the best results that can be directly fed into the `Trainer` component during model re-training.

In [33]:
tuner = Tuner(
        module_file=TRAINER_MODULE_FILE,
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        train_args=trainer_pb2.TrainArgs(num_steps=1000),
        eval_args=trainer_pb2.EvalArgs(num_steps=500))

In [34]:
context.run(tuner)

2023-10-20 15:10:34.072799: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type


1000/1000 [==============================] - ETA: 0s - loss: 2.0289 - sparse_categorical_accuracy: 0.171 - ETA: 2s - loss: 1.6468 - sparse_categorical_accuracy: 0.421 - ETA: 2s - loss: 1.5066 - sparse_categorical_accuracy: 0.486 - ETA: 2s - loss: 1.3710 - sparse_categorical_accuracy: 0.535 - ETA: 2s - loss: 1.2582 - sparse_categorical_accuracy: 0.564 - ETA: 2s - loss: 1.1736 - sparse_categorical_accuracy: 0.585 - ETA: 2s - loss: 1.1085 - sparse_categorical_accuracy: 0.601 - ETA: 2s - loss: 1.0650 - sparse_categorical_accuracy: 0.612 - ETA: 2s - loss: 1.0299 - sparse_categorical_accuracy: 0.619 - ETA: 2s - loss: 1.0014 - sparse_categorical_accuracy: 0.628 - ETA: 2s - loss: 0.9735 - sparse_categorical_accuracy: 0.633 - ETA: 2s - loss: 0.9532 - sparse_categorical_accuracy: 0.638 - ETA: 2s - loss: 0.9343 - sparse_categorical_accuracy: 0.643 - ETA: 2s - loss: 0.9190 - sparse_categorical_accuracy: 0.646 - ETA: 2s - loss: 0.9060 - sparse_categorical_accuracy: 0.648 - ETA: 2s - loss: 0.8960 - 

1000/1000 [==============================] - ETA: 0s - loss: 2.0277 - sparse_categorical_accuracy: 0.156 - ETA: 2s - loss: 1.2182 - sparse_categorical_accuracy: 0.537 - ETA: 2s - loss: 1.0141 - sparse_categorical_accuracy: 0.604 - ETA: 2s - loss: 0.9275 - sparse_categorical_accuracy: 0.634 - ETA: 2s - loss: 0.8748 - sparse_categorical_accuracy: 0.651 - ETA: 2s - loss: 0.8469 - sparse_categorical_accuracy: 0.659 - ETA: 2s - loss: 0.8260 - sparse_categorical_accuracy: 0.665 - ETA: 2s - loss: 0.8102 - sparse_categorical_accuracy: 0.670 - ETA: 2s - loss: 0.7975 - sparse_categorical_accuracy: 0.673 - ETA: 1s - loss: 0.7895 - sparse_categorical_accuracy: 0.674 - ETA: 1s - loss: 0.7793 - sparse_categorical_accuracy: 0.676 - ETA: 1s - loss: 0.7683 - sparse_categorical_accuracy: 0.679 - ETA: 1s - loss: 0.7574 - sparse_categorical_accuracy: 0.683 - ETA: 1s - loss: 0.7555 - sparse_categorical_accuracy: 0.682 - ETA: 1s - loss: 0.7521 - sparse_categorical_accuracy: 0.681 - ETA: 1s - loss: 0.7495 - 

/opt/conda/lib/python3.7/site-packages/sklearn/gaussian_process/kernels.py:427: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  ConvergenceWarning,


1000/1000 [==============================] - ETA: 0s - loss: 1.9904 - sparse_categorical_accuracy: 0.187 - ETA: 2s - loss: 1.6149 - sparse_categorical_accuracy: 0.442 - ETA: 2s - loss: 1.4446 - sparse_categorical_accuracy: 0.518 - ETA: 2s - loss: 1.3188 - sparse_categorical_accuracy: 0.563 - ETA: 2s - loss: 1.2201 - sparse_categorical_accuracy: 0.591 - ETA: 2s - loss: 1.1450 - sparse_categorical_accuracy: 0.608 - ETA: 2s - loss: 1.0902 - sparse_categorical_accuracy: 0.618 - ETA: 2s - loss: 1.0470 - sparse_categorical_accuracy: 0.626 - ETA: 2s - loss: 1.0140 - sparse_categorical_accuracy: 0.632 - ETA: 2s - loss: 0.9871 - sparse_categorical_accuracy: 0.639 - ETA: 2s - loss: 0.9634 - sparse_categorical_accuracy: 0.643 - ETA: 2s - loss: 0.9446 - sparse_categorical_accuracy: 0.647 - ETA: 1s - loss: 0.9262 - sparse_categorical_accuracy: 0.651 - ETA: 1s - loss: 0.9116 - sparse_categorical_accuracy: 0.654 - ETA: 1s - loss: 0.8983 - sparse_categorical_accuracy: 0.658 - ETA: 1s - loss: 0.8876 - 

...

1000/1000 [==============================] - ETA: 0s - loss: 2.0727 - sparse_categorical_accuracy: 0.078 - ETA: 3s - loss: 1.3453 - sparse_categorical_accuracy: 0.493 - ETA: 3s - loss: 1.0612 - sparse_categorical_accuracy: 0.594 - ETA: 2s - loss: 0.9334 - sparse_categorical_accuracy: 0.636 - ETA: 2s - loss: 0.8978 - sparse_categorical_accuracy: 0.641 - ETA: 2s - loss: 0.8708 - sparse_categorical_accuracy: 0.649 - ETA: 2s - loss: 0.8522 - sparse_categorical_accuracy: 0.656 - ETA: 2s - loss: 0.8362 - sparse_categorical_accuracy: 0.660 - ETA: 2s - loss: 0.8277 - sparse_categorical_accuracy: 0.662 - ETA: 2s - loss: 0.8137 - sparse_categorical_accuracy: 0.665 - ETA: 2s - loss: 0.8092 - sparse_categorical_accuracy: 0.666 - ETA: 2s - loss: 0.7984 - sparse_categorical_accuracy: 0.669 - ETA: 2s - loss: 0.7895 - sparse_categorical_accuracy: 0.671 - ETA: 2s - loss: 0.7835 - sparse_categorical_accuracy: 0.671 - ETA: 2s - loss: 0.7742 - sparse_categorical_accuracy: 0.674 - ETA: 2s - loss: 0.7683 - 

1000/1000 [==============================] - ETA: 0s - loss: 2.0639 - sparse_categorical_accuracy: 0.109 - ETA: 3s - loss: 1.3219 - sparse_categorical_accuracy: 0.520 - ETA: 3s - loss: 1.0884 - sparse_categorical_accuracy: 0.598 - ETA: 3s - loss: 0.9836 - sparse_categorical_accuracy: 0.622 - ETA: 3s - loss: 0.9272 - sparse_categorical_accuracy: 0.635 - ETA: 3s - loss: 0.8886 - sparse_categorical_accuracy: 0.647 - ETA: 3s - loss: 0.8626 - sparse_categorical_accuracy: 0.652 - ETA: 3s - loss: 0.8408 - sparse_categorical_accuracy: 0.660 - ETA: 3s - loss: 0.8243 - sparse_categorical_accuracy: 0.664 - ETA: 3s - loss: 0.8128 - sparse_categorical_accuracy: 0.666 - ETA: 2s - loss: 0.7991 - sparse_categorical_accuracy: 0.671 - ETA: 2s - loss: 0.7890 - sparse_categorical_accuracy: 0.673 - ETA: 2s - loss: 0.7832 - sparse_categorical_accuracy: 0.674 - ETA: 2s - loss: 0.7789 - sparse_categorical_accuracy: 0.675 - ETA: 2s - loss: 0.7737 - sparse_categorical_accuracy: 0.676 - ETA: 2s - loss: 0.7693 - 

INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


ExecutionResult(
    component_id: Tuner
    execution_id: 8
    outputs:
        best_hyperparameters: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 11
        type_id: 20
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Tuner/best_hyperparameters/8"
        custom_properties {
          key: "name"
          value {
            string_value: "best_hyperparameters"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Tuner"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "HyperParameters"
        )]
        ))

### Retrain your model by running Tuner with the best hyperparameters

In [35]:
hparams_importer = ImporterNode(
    instance_name='import_hparams',
    # This can be Tuner's output file or manually edited file. The file contains
    # text format of hyperparameters (kerastuner.HyperParameters.get_config())
    source_uri=tuner.outputs.best_hyperparameters.get()[0].uri,
    artifact_type=HyperParameters)

In [36]:
context.run(hparams_importer)

ExecutionResult(
    component_id: ImporterNode.import_hparams
    execution_id: 9
    outputs:
        result: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 11
        type_id: 20
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Tuner/best_hyperparameters/8"
        custom_properties {
          key: "name"
          value {
            string_value: "best_hyperparameters"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Tuner"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        create_time_since_epoch: 1697814634079
        last_update_time_since_epoch: 1697814703582
        , artifact_type: id: 20
        name: "HyperParameters"
        )]
        ))

In [37]:
# TODO: your code to retrain your model with the best hyperparameters found by the Tuner component above.
# Hint: review the Trainer code above in this notebook and the documentation for how to configure the trainer 
# to use the output artifact from the hparams_importer.

trainer = Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=TRAINER_MODULE_FILE,
    transformed_examples=transform.outputs.transformed_examples,
    schema=schema_importer.outputs.result,
    transform_graph=transform.outputs.transform_graph,
    hyperparameters=hparams_importer.outputs.result,    
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1000))

In [38]:
context.run(trainer)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
2023-10-20 15:14:57.001252: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.


   1/5000 [..............................] - ETA: 1s - loss: 2.0716 - sparse_categorical_accuracy: 0.1719

2023-10-20 15:14:59.973239: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.


   2/5000 [..............................] - ETA: 30:26 - loss: 1.9824 - sparse_categorical_accuracy: 0.1953WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0056s vs `on_train_batch_end` time: 0.7284s). Check your callbacks.


...

5000/5000 [==============================] - ETA: 5:26 - loss: 1.4286 - sparse_categorical_accuracy: 0.4896 - ETA: 3:00 - loss: 1.1634 - sparse_categorical_accuracy: 0.581 - ETA: 2:06 - loss: 1.0422 - sparse_categorical_accuracy: 0.609 - ETA: 1:38 - loss: 0.9782 - sparse_categorical_accuracy: 0.625 - ETA: 1:22 - loss: 0.9391 - sparse_categorical_accuracy: 0.636 - ETA: 1:11 - loss: 0.9007 - sparse_categorical_accuracy: 0.643 - ETA: 1:04 - loss: 0.8859 - sparse_categorical_accuracy: 0.645 - ETA: 59s - loss: 0.8762 - sparse_categorical_accuracy: 0.646 - ETA: 54s - loss: 0.8598 - sparse_categorical_accuracy: 0.64 - ETA: 50s - loss: 0.8483 - sparse_categorical_accuracy: 0.65 - ETA: 47s - loss: 0.8353 - sparse_categorical_accuracy: 0.65 - ETA: 44s - loss: 0.8212 - sparse_categorical_accuracy: 0.65 - ETA: 42s - loss: 0.8113 - sparse_categorical_accuracy: 0.66 - ETA: 40s - loss: 0.8035 - sparse_categorical_accuracy: 0.66 - ETA: 39s - loss: 0.7979 - sparse_categorical_accuracy: 0.66 - ETA: 38s 

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Trainer/model/10/serving_model_dir/assets


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Trainer/model/10/serving_model_dir/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 10
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 12
        type_id: 17
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Trainer/model/10"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "Model"
        )]
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 13
        type_id: 18
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Trainer/model_run/10"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "ModelRun"
        )]
        ))

## Evaluating trained models with Evaluator
The `Evaluator` component analyzes model performance using the [TensorFlow Model Analysis library](https://www.tensorflow.org/tfx/model_analysis/get_started). It runs inference requests on particular subsets of the test dataset, based on which slices are defined by the developer. Knowing which slices should be analyzed requires domain knowledge of what is important in this particular use case or domain. 

The `Evaluator` can also optionally validate a newly trained model against a previous model. In this lab, you only train one model, so the Evaluator automatically will label the model as "blessed".


<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Evaluator.png width="400">

### Configure and run the Evaluator component


Use the `ResolverNode` to pick the previous model to compare against.  The model resolver is only required if performing model validation in addition to evaluation. In this case we validate against the latest blessed model. If no model has been blessed before (as in this case) the evaluator will make our candidate the first blessed model.

In [39]:
model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))

In [40]:
context.run(model_resolver)

ExecutionResult(
    component_id: ResolverNode.latest_blessed_model_resolver
    execution_id: 11
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
        ))

Configure evaluation metrics and slices.

In [41]:
# TODO: Your code here to create a tfma.MetricThreshold. 
# Review the API documentation here: https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold
# Hint: Review the API documentation for tfma.GenericValueThreshold to constrain accuracy between 50% and 99%.

accuracy_threshold = tfma.MetricThreshold(
                value_threshold=tfma.GenericValueThreshold(
                    lower_bound={'value': 0.5},
                    upper_bound={'value': 0.99})
                )

metrics_specs = tfma.MetricsSpec(
                   metrics = [
                       tfma.MetricConfig(class_name='SparseCategoricalAccuracy',
                           threshold=accuracy_threshold),
                       tfma.MetricConfig(class_name='ExampleCount')])

eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='Cover_Type')
    ],
    metrics_specs=[metrics_specs],
    slicing_specs=[
        tfma.SlicingSpec(),
        tfma.SlicingSpec(feature_keys=['Wilderness_Area'])
    ]
)
eval_config

model_specs {
  label_key: "Cover_Type"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "Wilderness_Area"
}
metrics_specs {
  metrics {
    class_name: "SparseCategoricalAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.5
        }
        upper_bound {
          value: 0.99
        }
      }
    }
  }
  metrics {
    class_name: "ExampleCount"
  }
}

In [42]:
model_analyzer = Evaluator(
    examples=example_gen.outputs.examples,
    model=trainer.outputs.model,
    baseline_model=model_resolver.outputs.model,
    eval_config=eval_config
)

In [43]:
context.run(model_analyzer, enable_cache=False)

2023-10-20 15:17:55.845253: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
2023-10-20 15:17:55.849200: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
        please use "has_baseline" instead.


ExecutionResult(
    component_id: Evaluator
    execution_id: 12
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 14
        type_id: 23
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Evaluator/evaluation/12"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "ModelEvaluation"
        )]
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 15
        type_id: 24
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Evaluator/blessing/12"
        custom_properties {
          key: "blessed"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/home/jupyter/artifact-store/tfx-covertype-classifier/20231020_145654/Trainer/model/10"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 12
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "ModelBlessing"
        )]
        ))

### Check the model performance validation status

In [44]:
model_blessing_uri = model_analyzer.outputs.blessing.get()[0].uri
!ls -l {model_blessing_uri}

total 0
-rw-r--r-- 1 jupyter jupyter 0 Oct 20 15:18 NOT_BLESSED


### Visualize evaluation results
You can visualize the evaluation results using the `tfma.view.render_slicing_metrics()` function from TensorFlow Model Analysis library.

**Setup Note**: **Currently, TFMA visualizations don't render in  JupyterLab. Make sure that you run this notebook in Classic Notebook.**

In [45]:
evaluation_uri = model_analyzer.outputs['evaluation'].get()[0].uri
evaluation_uri
!ls {evaluation_uri}

eval_config.json  metrics  plots  validations


In [46]:
eval_result = tfma.load_eval_result(evaluation_uri)
eval_result

EvalResult(slicing_metrics=[((), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.7067698836326599}, 'example_count': {'doubleValue': 20148.0}}}}), ((('Wilderness_Area', 'Cache'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.480355829000473}, 'example_count': {'doubleValue': 1349.0}}}}), ((('Wilderness_Area', 'Commanche'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.7003179788589478}, 'example_count': {'doubleValue': 8806.0}}}}), ((('Wilderness_Area', 'Rawah'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.7630903124809265}, 'example_count': {'doubleValue': 8957.0}}}}), ((('Wilderness_Area', 'Neota'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.5694980621337891}, 'example_count': {'doubleValue': 1036.0}}}})], plots=[((), None), ((('Wilderness_Area', 'Cache'),), None), ((('Wilderness_Area', 'Commanche'),), None), ((('Wilderness_Area', 'Rawah'),), None), ((('Wilderness_Area', 'Neota'),), None)], config=model_

In [47]:
tfma.view.render_slicing_metrics(eval_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [48]:
tfma.view.render_slicing_metrics(
    eval_result, slicing_column='Wilderness_Area')

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Wilderness_Area:Cach…

## InfraValidator

The `InfraValidator` component acts as an additional early warning layer by validating a candidate model in a sandbox version of its serving infrastructure to prevent an unservable model from being pushed to production. Compared to the `Evaluator` component above which validates a model's performance, the `InfraValidator` component is validating that a model is able to generate predictions from served examples in an environment configured to match production. The config below takes a model and examples, launches the model in a sand-boxed [TensorflowServing](https://www.tensorflow.org/tfx/guide/serving) model server from the latest image in a local docker engine, and optionally checks that the model binary can be loaded and queried before "blessing" it for production.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/InfraValidator.png width="400">

In [49]:
infra_validator = InfraValidator(
    model=trainer.outputs['model'],
    examples=example_gen.outputs['examples'],
    serving_spec=infra_validator_pb2.ServingSpec(
        tensorflow_serving=infra_validator_pb2.TensorFlowServing(
            tags=['latest']),
      local_docker=infra_validator_pb2.LocalDockerConfig(),
  ),
    validation_spec=infra_validator_pb2.ValidationSpec(
        max_loading_time_seconds=60,
        num_tries=5,
    ),    
  request_spec=infra_validator_pb2.RequestSpec(
      tensorflow_serving=infra_validator_pb2.TensorFlowServingRequestSpec(),
          num_examples=5,
      )
)

In [ ]:
context.run(infra_validator, enable_cache=False)

2023-10-20 15:19:07.658956: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type


**Note**: In case of `Infra Validation (attempt 1/5) failed` error, please disregard the error.

### Check the model infrastructure validation status

In [ ]:
infra_blessing_uri = infra_validator.outputs.blessing.get()[0].uri
!ls -l {infra_blessing_uri}

## Deploying models with Pusher

The `Pusher` component checks whether a model has been "blessed", and if so, deploys it by pushing the model to a well known file destination.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Pusher.png width="400">



### Configure and run the `Pusher` component

In [ ]:
trainer.outputs['model']

In [ ]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=model_analyzer.outputs['blessing'],
    infra_blessing=infra_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR)))
context.run(pusher)

### Examine the output of `Pusher`

In [ ]:
pusher.outputs

In [ ]:
# Set `PATH` to include a directory containing `saved_model_cli.
PATH=%env PATH
%env PATH=/opt/conda/envs/tfx/bin:{PATH}

In [ ]:
latest_pushed_model = os.path.join(SERVING_MODEL_DIR, max(os.listdir(SERVING_MODEL_DIR)))
!saved_model_cli show --dir {latest_pushed_model} --all

**Note**: In case of `FileNotFoundError` error, please disregard the error.

## Next steps

This concludes your introductory walthrough through TFX pipeline components. In the lab, you used TFX to analyze, understand, and pre-process the dataset and train, analyze, validate, and deploy a multi-class classification model to predict the type of forest cover from cartographic features. You utilized a TFX Interactive Context for prototype development of a TFX pipeline directly in a Jupyter notebook. Next, you worked with the TFDV library to modify your dataset schema to add feature constraints to catch data anamolies that can negatively impact your model's performance. You utilized TFT library for feature proprocessing for consistent feature transformations for your model at training and serving time. Lastly, using the TFMA library, you added model performance constraints to ensure you only push more accurate models than previous runs to production.


The next labs in the series will guide through developing a TFX pipeline, deploying and running the pipeline on **AI Platform Pipelines** and automating the pipeline build and deployment processes with **Cloud Build**.

## License

<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>